In [20]:
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()

db_path = os.getenv("DB_PATH")

df = pd.read_parquet(db_path, engine="fastparquet")

In [21]:
df[df["wave"] == "2025-02"]["question"].nunique()

584

In [28]:
df[df["question"].str.contains("[Q3] В каком Федеральном Округе Вы живете?", regex=False)]

,wave,respondent_id,respondent_uid,question,answer
6030,2015-01,34764543,395220888,[Q3] В каком Федеральном Округе Вы живете?,Северо-Западном
6031,2015-02,41668184,395181661,[Q3] В каком Федеральном Округе Вы живете?,Северо-Западном
6032,2015-01,34320332,395225983,[Q3] В каком Федеральном Округе Вы живете?,Северо-Западном
6033,2015-02,40813067,395178351,[Q3] В каком Федеральном Округе Вы живете?,Северо-Западном
6034,2015-02,41626048,395179355,[Q3] В каком Федеральном Округе Вы живете?,Северо-Западном
...,...,...,...,...,...
25853314,2025-03,6340,11113134709,[Q3] В каком Федеральном Округе Вы живете?,Уральском
25853315,2025-03,6349,11113726151,[Q3] В каком Федеральном Округе Вы живете?,Центральном
25853316,2025-03,6358,11196235298,[Q3] В каком Федеральном Округе Вы живете?,Уральском
25853317,2025-03,6464,11197759939,[Q3] В каком Федеральном Округе Вы живете?,Уральском


In [2]:
from utils import get_unique_questions_info, get_question_parts

get_unique_questions_info(df)

,q_clean,waves,answers
0,Banks_number,"[2024-03, 2025-02]","[1, 10, 11, 12, 13, 15, 2, 3, 4, 5, 6, 7, 8, 9]"
1,Cервис и удобство покупки (наличие полной и до...,[2024-02],[Прочие оффлайн магазины]
2,Cервис и удобство покупки (наличие полной и до...,[2024-02],[Прочие онлайн магазины]
3,Cервис и удобство покупки (наличие полной и до...,[2024-02],[Все инструменты]
4,Cервис и удобство покупки (наличие полной и до...,[2024-02],[ЯндексМаркет]
...,...,...,...
3931,Я обычно покупаю обувь:,[2017-03],"[В первом же магазине, Я обычно захожу в 2-3 м..."
3932,Являетесь ли Вы подписчиком Сбера?,"[2021-03, 2021-04, 2022-01, 2022-03, 2022-04, ...","[Да, подписан на СберПрайм, Да, подписан на Сб..."
3933,Являются ли путешествия для вас источником сча...,[2025-02],"[Да, Нет]"
3934,Яндекс.Браузер уже был установлен на вашем тел...,[2017-02],"[Был установлен, Сам устанавливал]"


In [ ]:
qs_ans = df.groupby("question", observed=True).agg(
        waves=("wave", lambda x: set(x)),
        answers=("answer", lambda x: set(x))
    ).reset_index()

# qs_ans = pd.concat([
#     get_question_parts(qs_ans["question"]),
#     qs_ans.drop(columns=["question"])
# ], axis=1)

# qs_ans = qs_ans.groupby(["q_clean"], observed=True).agg(
#     waves   = ("waves",   lambda s: sorted(reduce(set.union, s, set()))),
#     answers = ("answers", lambda s: take_first_n(reduce(set.union, s, set())))
# ).reset_index()

In [ ]:
qs_ans["question"]

In [ ]:
get_question_parts(qs_ans["question"])

In [30]:
from capability_spec import CapabilitySpec
from planner import PlannerOut

In [40]:
# PlannerOut.model_json_schema()
import json
from pydantic import BaseModel
from typing import List, Optional, Dict, Any

class SimplePlanStep(BaseModel):
    id: str
    operation: str  # "LOAD_DATA", "FILTER", "PIVOT"
    goal: Optional[str] = ""
    inputs: Optional[Dict[str, Any]] = None
    outputs: Optional[List[str]] = None
    constraints: Optional[Dict[str, Any]] = None
    depends_on: Optional[List[str]] = None

class SimplePlannerOut(BaseModel):
    steps: List[SimplePlanStep]
    analysis: Optional[str] = ""

# Для промпта используем упрощенную схему
SimplePlannerOut

# Source - https://stackoverflow.com/a
# Posted by Mahmoud Hussien Mohamed, modified by community. See post 'Timeline' for change history
# Retrieved 2025-11-07, License - CC BY-SA 4.0

def remove_defs_and_refs(schema: dict):
    schema = schema.copy()
    defs = schema.pop('$defs', {})

    def resolve(subschema):
        if isinstance(subschema, dict):
            ref = subschema.get('$ref', None)
            if ref:
                _def = ref.split('/')[-1]
                return resolve(defs[_def])
            return {
                _def: resolve(_ref)
                for _def, _ref in subschema.items()
            }
        if isinstance(subschema, list):
            return [resolve(ss) for ss in subschema]
        return subschema
    
    return resolve(schema)


schema = PlannerOut.model_json_schema()
inlined = remove_defs_and_refs(schema) # No `$defs` or `$ref`s
print(json.dumps(inlined, indent=2, ensure_ascii=False))

{
  "properties": {
    "analysis": {
      "default": "",
      "description": "Короткий комментарий стратегии",
      "title": "Analysis",
      "type": "string"
    },
    "steps": {
      "items": {
        "properties": {
          "id": {
            "description": "Уникальный идентификатор шага (s1, s2, ...)",
            "title": "Id",
            "type": "string"
          },
          "goal": {
            "default": "",
            "description": "Человекочитаемая цель шага",
            "title": "Goal",
            "type": "string"
          },
          "operation": {
            "enum": [
              "LOAD_DATA",
              "FILTER",
              "PIVOT"
            ],
            "title": "OperationType",
            "type": "string"
          },
          "inputs": {
            "anyOf": [
              {
                "additionalProperties": true,
                "type": "object"
              },
              {
                "items": {},
                "typ

In [29]:
# print(CapabilitySpec().to_prompt_context("detailed"))

In [ ]:
# Получаем упрощенную схему без $defs
simple_schema = PlannerOut.model_json_schema(mode='serialization')
simple_schema.pop('$defs', None)

# Рекурсивно удаляем все $ref
def remove_refs(schema):
    if isinstance(schema, dict):
        schema.pop('$ref', None)
        for key, value in schema.items():
            remove_refs(value)
    elif isinstance(schema, list):
        for item in schema:
            remove_refs(item)
    return schema

remove_refs(simple_schema)

In [ ]:
from pydantic.json_schema import DEFAULT_REF_TEMPLATE
DEFAULT_REF_TEMPLATE

In [ ]:
PlannerOut.model_json_schema(ref_template='#/$defs/{model}')

In [ ]:
from schemas import OperationType

In [ ]:
list(OperationType.__members__.keys())